In [3]:
import threading
import time
# define rooms and items

couch = {
    "name": "couch",
    "type": "furniture",
}

piano = {
    "name": "piano",
    "type": "furniture",
}

queen_bed = {
    "name": "queenbed",
    "type": "furniture",
}

double_bed = {
    "name": "doublebed",
    "type": "furniture",
}

dresser = {
    "name": "dresser",
    "type": "furniture",
}

dining_table = {
    "name": "diningtable",
    "type": "furniture",
}


door_a = {
    "name": "door a",
    "type": "door",
}

door_b = {
    "name": "door b",
    "type": "door",
}

door_c = {
    "name": "door c",
    "type": "door",
}

door_d = {
    "name": "door d",
    "type": "door",
}



key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}

key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
}

key_c = {
    "name": "key for door c",
    "type": "key",
    "target": door_c,
}

key_d = {
    "name": "key for door d",
    "type": "key",
    "target": door_d,
}


game_room = {
    "name": "gameroom",
    "type": "room",
}


bedroom_1 = {
    "name": "bedroom 1",
    "type": "room",
}

bedroom_2 = {
    "name": "bedroom 2",
    "type": "room",
}

living_room = {
    "name": "livingroom",
    "type": "room",
}

outside = {
  "name": "outside"
}



all_rooms = [game_room, outside,bedroom_1,bedroom_2,living_room] #list containing all rooms

all_doors = [door_a, door_b,door_c,door_d] #list containing all doors

all_keys = [key_a, key_b, key_c, key_d] #list containing all keys


#Dictionary defining relationships between rooms and items
object_relations = {
    "gameroom": [couch, piano, door_a],
    "bedroom 1": [queen_bed,door_a,door_b,door_c],
    "bedroom 2":[double_bed,dresser,door_b],
    "livingroom":[dining_table,door_d],
    "outside":[door_d],
    "couch":[],
    "piano": [key_a],
    "queenbed":[key_b],
    "doublebed":[key_c],
    "dresser":[key_d],
    "diningtable":[],
    "door b":[bedroom_1,bedroom_2],
    "door c":[bedroom_1,living_room],
    "door a": [game_room, bedroom_1],
    "door d":[living_room,outside]
}

#A dictionary representing the initial state of the game
INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside
}


def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Start the game
    """

    global time_up

    time_up = False
    # Start the timer thread
    timer_thread = threading.Thread(target=timer_function, daemon=True) #study this

    timer_thread.start()

    print("You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!")
    play_room(game_state["current_room"])


    if time_up:
        print("Game over! Time's up.")
    else:
        print("Congratulations! You escaped the room and completed the game.") ###added here


def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either
    explore (list all items in this room) or examine an item found here.
    """

    game_state["current_room"] = room
    while True:
        if time_up: ###

         ## print("Time's up!")###
          break
        elif(game_state["current_room"] == game_state["target_room"]):
          #print("Congrats! You escaped the room!")                       #removed this


          break
        else:
          print("You are now in " + room["name"])
          #items_in_room = [item["name"] for item in object_relations[room["name"]]] # creates a list of objects found ,iterating through every item in the list of objects associated with the current room , extracts the name of the objects
          #print("What you can find in this room:", items_in_room)

          intended_action = input("What would you like to do? Type 'explore' or 'examine'?").strip()
          if intended_action == "explore":
            explore_room(room)
            play_room(room)
          elif intended_action == "examine":
            examine_item(input("What would you like to examine?").strip())
          else:
            print("Not sure what you mean. Type 'explore' or 'examine'.")
            play_room(room)
        linebreak()

#Timer function
#####
def timer_function():
    """Function to run as a timer"""
    global time_up

    time_left = 150 # setting the timer duration in seconds

    while time_left > 0:
        print(f"Time left: {time_left} SECONDS!")
        time.sleep(30)  #pauses thread
        time_left -= 30 #decrements
    time_up = True
   ############### #print("Time's up!")

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]] # creates a list of objects found ,iterating through every item in the list of objects associated with the current room , extracts the name of the objects
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    #Handling  the errors appropriately
    try:
      connected_rooms = object_relations[door["name"]]
      next_room = next((room for room in connected_rooms if not current_room == room), None)
      return next_room
    except KeyError:
      print("Error: Door not found in object_relations.")
      # Handle the error appropriately, such as returning None or raising an exception
      return None


    #connected_rooms = object_relations[door["name"]]
    #for room in connected_rooms:
      #  if(not current_room == room):
        #    return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None

    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = "You examine " + item_name + ". "
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    output += "You unlock it with a key you have."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key."
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You find " + item_found["name"] + "."
                else:
                    output += "There isn't anything interesting about it."
            print(output)
            break

    if(output is None):
        print("The item you requested is not found in the current room.")

    if(next_room and input("Do you want to go to the next room? Enter 'yes' or 'no'").strip() == 'yes'):
        play_room(next_room)
    else:
        play_room(current_room)

In [4]:



#i placed this here because this will go to our main file ---> main.ipynb
#all the func will be in another file ----> escaperoomfunc.py

#import escaperoomfunc

game_state = INIT_GAME_STATE.copy()
start_game()

Time left: 150 SECONDS!
You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!
You are now in gameroom
What would you like to do? Type 'explore' or 'examine'?examine
Time left: 120 SECONDS!
Time left: 90 SECONDS!
Time left: 60 SECONDS!
Time left: 30 SECONDS!
What would you like to examine?piano
You examine piano. You find key for door a.



Game over! Time's up.
